In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import koreanize_matplotlib
from datetime import datetime, timedelta
import time
import dateutil
from pathlib import Path

In [2]:
PATH = r"C:\Users\kmfm1\BIproject\data"

In [3]:
customer_feedback = pd.read_csv(PATH + r"\customer_feedback.csv")
customers = pd.read_csv(PATH + r"\customers.csv")
delivery_performance = pd.read_csv(PATH + r"\delivery_performance.csv")
inv = pd.read_csv(PATH + r"\inventory.csv")
inventoryNew = pd.read_csv(PATH + r"\inventoryNew.csv")
order_items = pd.read_csv(PATH + r"\order_items.csv")
orders = pd.read_csv(PATH + r"\Business.csv")
products = pd.read_csv(PATH + r"\products.csv")
marketing_performance = pd.read_csv(PATH + r"\marketing_performance.csv")

In [4]:
# =========================
# 1) Parse dates + week_of_month (1~5)
#    - 주차 정의: 1~7일=1주차, 8~14일=2주차 ...
# =========================
inv["date"] = pd.to_datetime(inv["date"], format="%d-%m-%Y", errors="coerce")
orders["order_date"] = pd.to_datetime(orders["order_date"], errors="coerce")

inv = inv.dropna(subset=["date"])
orders = orders.dropna(subset=["order_date"])

def add_y_m_w(df, date_col):
    df = df.copy()
    df["year"] = df[date_col].dt.year.astype(int)
    df["month"] = df[date_col].dt.month.astype(int)
    df["week"] = ((df[date_col].dt.day - 1) // 7 + 1).astype(int)
    return df

inv = add_y_m_w(inv, "date")
orders = add_y_m_w(orders, "order_date")

In [5]:
# =========================
# 2) Weekly inbound (stock_received / damaged_stock)
# =========================
weekly_in = (
    inv.groupby(["product_id", "year", "month", "week"], as_index=False)
       .agg(
           stock_received_week=("stock_received", "sum"),
           damaged_week=("damaged_stock", "sum"),
       )
)
weekly_in

,product_id,year,month,week,stock_received_week,damaged_week
0,4452,2023,3,3,0,6
1,4452,2023,3,4,6,4
2,4452,2023,4,1,3,2
3,4452,2023,4,2,15,4
4,4452,2023,4,3,3,4
...,...,...,...,...,...,...
25067,993331,2024,10,2,6,2
25068,993331,2024,10,3,4,4
25069,993331,2024,10,4,4,4
25070,993331,2024,10,5,3,2


In [6]:
# =========================
# 3) Weekly sales qty (orders + order_items)
# =========================
# NOTE: order_items에 product_id, quantity가 있어야 정상 계산됨
oi = orders[["order_id", "year", "month", "week"]].merge(
    order_items[["order_id", "product_id", "quantity"]],
    on="order_id",
    how="inner",
)

weekly_sales = (
    oi.groupby(["product_id", "year", "month", "week"], as_index=False)
      .agg(sold_qty_week=("quantity", "sum"))
)
weekly_sales

,product_id,year,month,week,sold_qty_week
0,4452,2023,4,1,3
1,4452,2023,4,4,5
2,4452,2023,6,1,2
3,4452,2023,6,2,1
4,4452,2023,6,4,3
...,...,...,...,...,...
4535,993331,2024,3,4,1
4536,993331,2024,5,1,1
4537,993331,2024,5,3,2
4538,993331,2024,6,2,2


In [7]:
# =========================
# 4) Full grid 만들기 (입고 없는 주에도 판매만 있는 주가 누락되지 않게)
#    - "등장한 주차"들(입고/판매) + 제품들로 grid
# =========================
weeks_union = pd.concat([
    weekly_in[["year", "month", "week"]],
    weekly_sales[["year", "month", "week"]],
], ignore_index=True).drop_duplicates()

products_union = pd.concat([
    weekly_in[["product_id"]],
    weekly_sales[["product_id"]],
], ignore_index=True).drop_duplicates()

# cross join
weeks_union["_k"] = 1
products_union["_k"] = 1
grid = products_union.merge(weeks_union, on="_k", how="outer").drop(columns="_k")

In [8]:
# =========================
# 5) grid에 입고/판매 붙이기
# =========================
weekly = grid.merge(
    weekly_in,
    on=["product_id", "year", "month", "week"],
    how="left",
).merge(
    weekly_sales,
    on=["product_id", "year", "month", "week"],
    how="left",
)

weekly["stock_received_week"] = weekly["stock_received_week"].fillna(0).astype(float)
weekly["damaged_week"] = weekly["damaged_week"].fillna(0).astype(float)
weekly["sold_qty_week"] = weekly["sold_qty_week"].fillna(0).astype(float)

# 정렬 (누적 계산 필수)
weekly = weekly.sort_values(["product_id", "year", "month", "week"]).reset_index(drop=True)

In [9]:
# =========================
# 6) 누적 재고 계산 (초기 0)
#    - 네가 말한 방식: current_inventory = cum_stock - cum_sold
#    - damaged는 별도 누적만: cum_damaged (원하면 차감 버전도 같이 제공)
# =========================
weekly["cum_stock"] = weekly.groupby("product_id")["stock_received_week"].cumsum()
weekly["cum_sold"] = weekly.groupby("product_id")["sold_qty_week"].cumsum()
weekly["cum_damaged"] = weekly.groupby("product_id")["damaged_week"].cumsum()

# (A) 네 방식: damaged 미차감 "장부상 재고"
weekly["current_inventory"] = weekly["cum_stock"] - weekly["cum_sold"]

# (B) 참고: damaged 차감한 "가용 재고"도 같이 만들어두기
weekly["available_inventory"] = weekly["cum_stock"] - weekly["cum_sold"] - weekly["cum_damaged"]

# 주차 라벨
weekly["week_label"] = (
    weekly["year"].astype(str)
    + "-"
    + weekly["month"].astype(str).str.zfill(2)
    + " "
    + weekly["week"].astype(str)
    + "주차"
)

In [10]:
# =========================
# 7) product_name 붙이기
# =========================
weekly = weekly.merge(
    products[["product_id", "product_name", "category", "brand"]],
    on="product_id",
    how="left",
)
weekly

,product_id,year,month,week,stock_received_week,damaged_week,sold_qty_week,cum_stock,cum_sold,cum_damaged,current_inventory,available_inventory,week_label,product_name,category,brand
0,4452,2023,3,3,0.0,6.0,0.0,0.0,0.0,6.0,0.0,-6.0,2023-03 3주차,Baby Wipes,Baby Care,Morar-Mistry
1,4452,2023,3,4,6.0,4.0,0.0,6.0,0.0,10.0,6.0,-4.0,2023-03 4주차,Baby Wipes,Baby Care,Morar-Mistry
2,4452,2023,3,5,0.0,0.0,0.0,6.0,0.0,10.0,6.0,-4.0,2023-03 5주차,Baby Wipes,Baby Care,Morar-Mistry
3,4452,2023,4,1,3.0,2.0,3.0,9.0,3.0,12.0,6.0,-6.0,2023-04 1주차,Baby Wipes,Baby Care,Morar-Mistry
4,4452,2023,4,2,15.0,4.0,0.0,24.0,3.0,16.0,21.0,5.0,2023-04 2주차,Baby Wipes,Baby Care,Morar-Mistry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26527,993331,2024,10,2,6.0,2.0,0.0,444.0,23.0,292.0,421.0,129.0,2024-10 2주차,Pulses,Grocery & Staples,"Parmer, Gola and Yogi"
26528,993331,2024,10,3,4.0,4.0,0.0,448.0,23.0,296.0,425.0,129.0,2024-10 3주차,Pulses,Grocery & Staples,"Parmer, Gola and Yogi"
26529,993331,2024,10,4,4.0,4.0,0.0,452.0,23.0,300.0,429.0,129.0,2024-10 4주차,Pulses,Grocery & Staples,"Parmer, Gola and Yogi"
26530,993331,2024,10,5,3.0,2.0,0.0,455.0,23.0,302.0,432.0,130.0,2024-10 5주차,Pulses,Grocery & Staples,"Parmer, Gola and Yogi"


In [11]:
# =========================
# 8) 최종 컬럼 선택
# =========================
result = weekly[[
    "week_label", "year", "month", "week",
    "product_id", "product_name", "category", "brand",
    "stock_received_week", "sold_qty_week", "damaged_week",
    "cum_stock", "cum_sold", "cum_damaged",
    "current_inventory", "available_inventory",
]]
result[result['week_label']=='2023-03 3주차']

,week_label,year,month,week,product_id,product_name,category,brand,stock_received_week,sold_qty_week,damaged_week,cum_stock,cum_sold,cum_damaged,current_inventory,available_inventory
0,2023-03 3주차,2023,3,3,4452,Baby Wipes,Baby Care,Morar-Mistry,0.0,0.0,6.0,0.0,0.0,6.0,0.0,-6.0
99,2023-03 3주차,2023,3,3,6405,Baby Food,Baby Care,Kashyap-Reddy,6.0,0.0,4.0,6.0,0.0,4.0,6.0,2.0
198,2023-03 3주차,2023,3,3,9436,Toothpaste,Personal Care,Naidu PLC,4.0,2.0,4.0,4.0,2.0,4.0,2.0,-2.0
297,2023-03 3주차,2023,3,3,11422,Potatoes,Fruits & Vegetables,Ramaswamy-Tata,8.0,0.0,2.0,8.0,0.0,2.0,8.0,6.0
396,2023-03 3주차,2023,3,3,14145,Spinach,Fruits & Vegetables,"Thakur, Shah and Pingle",3.0,3.0,4.0,3.0,3.0,4.0,0.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26037,2023-03 3주차,2023,3,3,970529,Diapers,Baby Care,"Kurian, Patla and Sanghvi",7.0,0.0,2.0,7.0,0.0,2.0,7.0,5.0
26136,2023-03 3주차,2023,3,3,990993,Chocolates,Snacks & Munchies,Guha-Subramaniam,2.0,0.0,2.0,2.0,0.0,2.0,2.0,0.0
26235,2023-03 3주차,2023,3,3,991434,Cookies,Snacks & Munchies,Mital-Oza,0.0,1.0,4.0,0.0,1.0,4.0,-1.0,-5.0
26334,2023-03 3주차,2023,3,3,992178,Pain Reliever,Pharmacy,Basu and Sons,3.0,0.0,4.0,3.0,0.0,4.0,3.0,-1.0


In [12]:
result.drop(columns=['available_inventory'], axis=1, inplace=True)

C:\Users\kmfm1\AppData\Local\Temp\ipykernel_24272\960867871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.drop(columns=['available_inventory'], axis=1, inplace=True)


In [13]:
result[result['product_name'] == 'Baby Wipes']

,week_label,year,month,week,product_id,product_name,category,brand,stock_received_week,sold_qty_week,damaged_week,cum_stock,cum_sold,cum_damaged,current_inventory
0,2023-03 3주차,2023,3,3,4452,Baby Wipes,Baby Care,Morar-Mistry,0.0,0.0,6.0,0.0,0.0,6.0,0.0
1,2023-03 4주차,2023,3,4,4452,Baby Wipes,Baby Care,Morar-Mistry,6.0,0.0,4.0,6.0,0.0,10.0,6.0
2,2023-03 5주차,2023,3,5,4452,Baby Wipes,Baby Care,Morar-Mistry,0.0,0.0,0.0,6.0,0.0,10.0,6.0
3,2023-04 1주차,2023,4,1,4452,Baby Wipes,Baby Care,Morar-Mistry,3.0,3.0,2.0,9.0,3.0,12.0,6.0
4,2023-04 2주차,2023,4,2,4452,Baby Wipes,Baby Care,Morar-Mistry,15.0,0.0,4.0,24.0,3.0,16.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24448,2024-10 2주차,2024,10,2,927651,Baby Wipes,Baby Care,Narula Group,0.0,0.0,2.0,584.0,35.0,346.0,549.0
24449,2024-10 3주차,2024,10,3,927651,Baby Wipes,Baby Care,Narula Group,0.0,0.0,2.0,584.0,35.0,348.0,549.0
24450,2024-10 4주차,2024,10,4,927651,Baby Wipes,Baby Care,Narula Group,10.0,0.0,2.0,594.0,35.0,350.0,559.0
24451,2024-10 5주차,2024,10,5,927651,Baby Wipes,Baby Care,Narula Group,0.0,1.0,0.0,594.0,36.0,350.0,558.0


In [14]:
stock_dashboard = result.to_csv('stock_dashboard.csv', index=False)